In [ ]:
from codecs import decode
import numpy as np
import struct
import math

def bin_to_float(bitstring):
    if len(bitstring) != 16 or not all(c in '01' for c in bitstring):
        raise ValueError("Invalid bitstring")
    
    sign = 1 if bitstring[0] == '0' else -1         
    exponent_bits = bitstring[1:6]                
    mantissa_bits = bitstring[6:16]               
    
    E = int(exponent_bits, 2)                        
    M = int(mantissa_bits, 2)                        
    
    if E == 0:
        if M == 0:
            return sign * 0.0                        
        else:
            return sign * (2 ** (-14)) * (M / 1024.0)
    elif E == 31:
        if M == 0:
            return sign * float('inf')               
        else:
            return float('nan')                     
    else:
        return sign * (2 ** (E - 15)) * (1 + M / 1024.0)
    
    
print(bin_to_float('0000000000000001'))

5.960464477539063e-08


In [ ]:
class Qubit():
    def __init__(self, measurement):
        self.measurement = None
        self.alpha = None
        self.beta = None
        self.superposition = None
        
        if (measurement is not None):
            self.measurement = measurement
            self.superposition = False
        
        
    def apply_hadamard(self):
        self.alpha = 1/np.sqrt(2)
        self.beta = 1/np.sqrt(2)
        self.superposition = True
        
        return self
        
        
    def apply_xgate(self):
        if (self.measurement == 0):
            self.measurement = 1
        else:
            self.measurement = 0
        
    def measure(self):
        if (self.superposition == False):
            return self.measurement
        
        var = np.random.uniform(0, 1)
        if np.abs(self.alpha) ** 2 >= var:
            self.measurement = 0
        else:
            self.measurement = 1
        self.superposition = False
        
        return self.measurement
    
    def __str__(self):
        return f'Qubit[superposition: {self.superposition}, [measurement: {self.measurement}]]'


def single_qubit_measurement(qubit):
    return qubit.measure()


def full_measurement(qc, nq):
    c_new = [0] * nq
    for l in range(nq):
        c_new[l] = str(qc[l].measure())
    
    return ''.join(c_new)



def crossover(q, m, a):
    q_0 = []
    for qubit in q:
        new_qubit = Qubit(0)
        new_qubit.alpha = qubit.alpha
        new_qubit.beta = qubit.beta
        new_qubit.measurement = qubit.measurement
        new_qubit.superposition = qubit.superposition
        q_0.append(new_qubit)
        
    r = np.random.randint(0, len(q) - a)
    
    for k in range(r, r + a):
        if q[k].measurement == m[k].measurement:
            q_0[k].apply_hadamard()
        else:
            q_0[k].apply_xgate()
    
    c_new = full_measurement(q_0, len(q_0))
    
    return c_new

def fitness_first(x):
    return (x - 15) ** 2 

def fitness_second(x):
    if not -5 <= x <= 5:
        return float('inf')  
    
    A = 10
    rastrigin = A + (x**2 - A * math.cos(2 * math.pi * x))
    
    discontinuity = 50 if -0.1 < x < 0.1 else 0
    
    noise = np.random.uniform(-0.5, 0.5)
    
    return rastrigin + discontinuity + noise

def deceptive_function(x):
    if not -10 <= x <= 10:
        return float('inf')  
    
    schwefel = 418.9829 - x * math.sin(math.sqrt(abs(x)))
    
    plateau = 100 if -2 <= x <= 2 else 0
    dip = -50 if -0.1 < x < 0.1 else 0
    
    poly = 0.1 * x**4 - 2 * x**2
    
    return schwefel + plateau + dip + poly

def evolution(population_size, fitness, qubits_in_indiv = 16 ,num_males = 20, num_elites = 20, max_iteration = 500, crossover_size = 3):
        
    population = []
    for i in range(population_size):
        q_0 = [Qubit(0) for _ in range(qubits_in_indiv)]
        q_0 = [qubit.apply_hadamard() for qubit in q_0]
        q = [str(qubit.measure()) for qubit in q_0]
        c_i = ''.join(q)
        population.append(c_i)
        
    population.sort(key = lambda x: fitness(bin_to_float(x))) # descending order if maximize 
    
    for indiv in population:
        print(bin_to_float(indiv))
    
    queen = population[0]
    males = population[1: num_males ]
    remaining = population[num_males:]
    # elite = population[:num_elites]
    for _ in range(max_iteration):
        for l in range(population_size - num_elites, population_size):
            population[l] = crossover([Qubit(int(char)) for char in queen], [Qubit(int(char)) for char in males[np.random.randint(0, num_males - 1)]], crossover_size)
            
        population.sort(key = lambda x: fitness(bin_to_float(x))) # descending order if maximize 
        queen = population[0]
        males = population[1: num_males ]
        print(f'Iteration: {_}, Queen: {bin_to_float(queen)}, Fitness: {fitness(bin_to_float(queen))}')

    return bin_to_float(queen)
    

evolution(1000, fitness=deceptive_function)

3.1875
3.078125
2.94921875
2.90625
2.875
2.8359375
2.744140625
3.796875
2.328125
-3.0546875
-3.017578125
-3.0
-2.955078125
-3.23828125
-2.814453125
-3.275390625
-2.7578125
-2.578125
-3.556640625
-2.416015625
-2.412109375
-2.392578125
-3.599609375
-2.267578125
-2.216796875
-3.703125
-2.19140625
-2.1484375
4.54296875
4.6015625
-4.20703125
-4.2734375
-4.3515625
4.7734375
-4.640625
-4.76953125
5.1015625
-4.8359375
-4.91796875
-5.015625
-5.11328125
5.39453125
-5.21875
-5.23828125
-5.265625
-5.3125
-5.35546875
5.60546875
-5.7734375
0.09576416015625
0.08868408203125
0.07080078125
0.0650634765625
0.062225341796875
0.061370849609375
0.06005859375
0.059600830078125
0.055419921875
0.053680419921875
0.051727294921875
0.047027587890625
0.04327392578125
0.042327880859375
0.0347900390625
0.03472900390625
0.032928466796875
0.030853271484375
0.0301055908203125
0.0296783447265625
0.029327392578125
0.0274505615234375
0.024322509765625
0.0236968994140625
0.021484375
0.0175323486328125
0.0169677734375
0.01

3.25390625